# Lab | Data Structuring and Combining Data

Objective: 
- Combine and integrate data from multiple sources using merging, concatenating, or joining techniques to generate more comprehensive and meaningful datasets for analysis.
- Modify the structure of data by pivoting, stacking/unstacking, or melting dataframes, enabling them to efficiently explore and analyze complex datasets. 

# Challenge 1: Combining Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the following links: 
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv


Perform data cleaning and formatting using the main cleaning and formatting function created in the previous lab to clean and format the data.

Combine the data from the three dataframes into a single dataframe, named "customer_data", using appropriate merging, concatenating, and joining techniques.

Verify that the customer_data dataframe contains all the rows and columns from the three original dataframes.

Observation: 
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [1]:
# Your code here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
# Read the data
df1_cleaned = pd.read_csv("Week2_2023.06.05_file1_cleaned.csv")
df2 = pd.read_csv("Week2_2023.06.06_Lab1_file2.csv")
df3 = pd.read_csv("Week2_2023.06.06_Lab1_file3.csv")

#print(df1_cleaned.head())
#print(df2.tail())
#print(df3.tail())

print(df1_cleaned.columns)
print(df2.columns)
print(df3.columns)

Index(['customer', 'st', 'gender', 'education', 'customer lifetime value',
       'income', 'monthly premium auto', 'number of open complaints',
       'policy type', 'vehicle class', 'total claim amount'],
      dtype='object')
Index(['Customer', 'ST', 'GENDER', 'Education', 'Customer Lifetime Value',
       'Income', 'Monthly Premium Auto', 'Number of Open Complaints',
       'Total Claim Amount', 'Policy Type', 'Vehicle Class'],
      dtype='object')
Index(['Customer', 'State', 'Customer Lifetime Value', 'Education', 'Gender',
       'Income', 'Monthly Premium Auto', 'Number of Open Complaints',
       'Policy Type', 'Total Claim Amount', 'Vehicle Class'],
      dtype='object')


In [9]:
#df1_cleaned = df1_cleaned.drop(df1_cleaned.columns[[0, 1]], axis=1, inplace=True)
#print(df1_cleaned.head())

# Rename columns
df2.columns=[i.lower() for i in df2.columns]
df3.columns=[i.lower() for i in df3.columns]

df1_cleaned.rename(columns={'st':'state'}, inplace=True)
df2.rename(columns={'st':'state'}, inplace=True)

#print(df1_cleaned.columns)
#print(df2.columns)
#print(df3.columns)

print(df1_cleaned.shape)
print(df2.shape)
print(df3.shape)

(913, 11)
(996, 11)
(7070, 11)


In [10]:
# Homogenize State column
def fixState (df):
    df["state"] = df["state"].replace({"Cali": "California"})
    df["state"] = df["state"].replace({"WA": "Washington"})
    df["state"] = df["state"].replace({"AZ": "Arizona"})
    return df
    
df2 = fixState(df2)
df3 = fixState(df3)

In [11]:
# Fix "Number of Open Complaints" column
def fixNumberOfOpenCompaints (df):
    if len(str(df["number of open complaints"][2])) > 1:
        df["number of open complaints"] = df["number of open complaints"].apply(lambda x:int(x[2]))
    return df

df2 = fixNumberOfOpenCompaints(df2)
df3 = fixNumberOfOpenCompaints(df3)

In [12]:
print(df1_cleaned["number of open complaints"].unique())
print(df2["number of open complaints"].unique())
print(df3["number of open complaints"].unique())

[0 1 2 4 3 5]
[0 1 3 5 2 4]
[0 2 3 1 5 4]


In [195]:
def changeDataTypesOfColumnsAndDropDupAndNA (df):
    # change column data types
    df['customer'] = pd.Categorical(df.customer)
    df['state'] = pd.Categorical(df.state)
    df['gender'] = pd.Categorical(df.gender)
    df['education'] = pd.Categorical(df.education)
    df['policy type'] = pd.Categorical(df["policy type"])
    df['vehicle class'] = pd.Categorical(df["vehicle class"])
    df['number of open complaints'] = df['number of open complaints'].astype(dtype = 'int64')
    df['customer lifetime value'] = df['customer lifetime value'].astype(dtype = 'string')
    df['customer lifetime value'] = df['customer lifetime value'].str.replace("%", "")
    df['customer lifetime value'] = df['customer lifetime value'].astype(dtype = 'float64')
    df['monthly premium auto'] = df['monthly premium auto'].astype(dtype = 'float64')
    df['income'] = df['income'].astype(dtype = 'float64')
    df['total claim amount'] = df['total claim amount'].astype(dtype = 'float64')
    
    # drop nas
    df = df.dropna()
    # drop duplicates
    df = df.drop_duplicates(keep="first")
    # RESET INDEX AFTER DROPPING ANY ROWS
    df = df.reset_index()
    return df

df1_cleaned = changeDataTypesOfColumnsAndDropDupAndNA(df1_cleaned)
df2 = changeDataTypesOfColumnsAndDropDupAndNA(df2)
df3 = changeDataTypesOfColumnsAndDropDupAndNA(df3)

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customer                   996 non-null    object 
 1   state                      996 non-null    object 
 2   gender                     991 non-null    object 
 3   education                  996 non-null    object 
 4   customer lifetime value    992 non-null    object 
 5   income                     996 non-null    int64  
 6   monthly premium auto       996 non-null    int64  
 7   number of open complaints  996 non-null    int64  
 8   total claim amount         996 non-null    float64
 9   policy type                996 non-null    object 
 10  vehicle class              996 non-null    object 
dtypes: float64(1), int64(3), object(7)
memory usage: 85.7+ KB


In [22]:
nanValues = df2.isnull()
duplicatedValues = df2.duplicated("customer lifetime value")

In [23]:
# print(nanValues.sum(axis=0))
print(duplicatedValues.value_counts())

False    951
True      45
dtype: int64


In [30]:
#print(df1_cleaned.shape)
#print(df2.shape)
#print(df3.shape)

#print(df1_cleaned.head())
#print(df2.tail())
df3.head()

,customer,state,customer lifetime value,education,gender,income,monthly premium auto,number of open complaints,policy type,total claim amount,vehicle class
0,SA25987,Washington,3479.137523,High School or Below,M,0,104,0,Personal Auto,499.200000,Two-Door Car
1,TB86706,Arizona,2502.637401,Master,M,0,66,0,Personal Auto,3.468912,Two-Door Car
2,ZL73902,Nevada,3265.156348,Bachelor,F,25820,82,0,Personal Auto,393.600000,Four-Door Car
3,KX23516,California,4455.843406,High School or Below,F,0,121,0,Personal Auto,699.615192,SUV
4,FN77294,California,7704.958480,High School or Below,M,30366,101,2,Personal Auto,484.800000,SUV


In [197]:
print(df1_cleaned.dtypes)
print(df2.dtypes)
print(df3.dtypes)

customer                     category
state                        category
gender                       category
education                    category
customer lifetime value       float64
income                        float64
monthly premium auto          float64
number of open complaints       int64
policy type                  category
vehicle class                category
total claim amount            float64
dtype: object
customer                     category
state                        category
gender                       category
education                    category
customer lifetime value       float64
income                        float64
monthly premium auto          float64
number of open complaints       int64
total claim amount            float64
policy type                  category
vehicle class                category
dtype: object
customer                     category
state                        category
customer lifetime value       float64
education             

In [198]:
#df_final = df1_cleaned.merge(df2)
#df_final = df2.merge(df3) #, left_on='customer', right_on='customer')
df_final = pd.concat([df1_cleaned, df2, df3]) #, ignore_index = True)

#df_final = df_final.merge(df3)

In [239]:
print(df_final.shape)

(8971, 11)


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [23]:
# Read the data
df4 = pd.read_csv("Week2_2023.06.06_Lab1_marketing_customer_analysis.csv")

#print(df1_cleaned.head())
#print(df2.tail())
#print(df3.tail())

print(df4.columns)
print(df4.dtypes)
print(df4.shape)

print(df4.head)

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')
Unnamed: 0                         int64
Customer                          object
State                             object
Customer Lifetime Value          float64
Response                          object
Coverage                          object
Education                         object
Effective To Date                 object
EmploymentStatus                  object
Gender                            object
Income                             int64
Locatio

## Exercise 1: Clean and Format the Data

While the dataset has been partially cleaned and formatted, we still need to perform several data cleaning tasks. Specifically, we need to standardize the column names, clean null values, convert the effective_to_date column to datetime, and extract the months from the dataset and store them in a separate column. 

To accomplish these tasks, we will use the functions created in the previous step to standardize the column names and deal with null values, and then we will apply additional functions to convert the effective_to_date column to datetime and extract the months.

Save the clean dataset into a csv file.

In [24]:
df4.columns=[i.lower() for i in df4.columns]
#df4_cleaned.rename(columns={'st':'state'}, inplace=True)

df4 = df4.drop("unnamed: 0",axis=1)
df4 = df4.drop("vehicle type",axis=1)

#print(df4.columns)
df4.columns

Index(['customer', 'state', 'customer lifetime value', 'response', 'coverage',
       'education', 'effective to date', 'employmentstatus', 'gender',
       'income', 'location code', 'marital status', 'monthly premium auto',
       'months since last claim', 'months since policy inception',
       'number of open complaints', 'number of policies', 'policy type',
       'policy', 'renew offer type', 'sales channel', 'total claim amount',
       'vehicle class', 'vehicle size'],
      dtype='object')

In [27]:
print(df4.isnull().sum(axis = 0))
# print(df4["vehicle type"].unique())

customer                           0
state                            631
customer lifetime value            0
response                         631
coverage                           0
education                          0
effective to date                  0
employmentstatus                   0
gender                             0
income                             0
location code                      0
marital status                     0
monthly premium auto               0
months since last claim          633
months since policy inception      0
number of open complaints          0
number of policies                 0
policy type                        0
policy                             0
renew offer type                   0
sales channel                      0
total claim amount                 0
vehicle class                    622
vehicle size                     622
dtype: int64


In [26]:
df4['number of open complaints'] = df4['number of open complaints'].fillna(0)

In [28]:
# turn data type to date
df4['effective to date'] = pd.to_datetime(df4['effective to date'])

In [29]:
print(df4.columns)
print(df4.dtypes)

Index(['customer', 'state', 'customer lifetime value', 'response', 'coverage',
       'education', 'effective to date', 'employmentstatus', 'gender',
       'income', 'location code', 'marital status', 'monthly premium auto',
       'months since last claim', 'months since policy inception',
       'number of open complaints', 'number of policies', 'policy type',
       'policy', 'renew offer type', 'sales channel', 'total claim amount',
       'vehicle class', 'vehicle size'],
      dtype='object')
customer                                 object
state                                    object
customer lifetime value                 float64
response                                 object
coverage                                 object
education                                object
effective to date                datetime64[ns]
employmentstatus                         object
gender                                   object
income                                    int64
location code  

In [30]:
# extract month
df4['month'] = df4['effective to date'].dt.month

In [31]:
df4.iloc[:5,:]

,customer,state,customer lifetime value,response,coverage,education,effective to date,employmentstatus,gender,income,...,number of open complaints,number of policies,policy type,policy,renew offer type,sales channel,total claim amount,vehicle class,vehicle size,month
0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,48029,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,2
1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,0,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,1
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,2
3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,1
4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,...,0.0,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,1


In [12]:
df4.head()

,customer,state,customer lifetime value,response,coverage,education,effective to date,employmentstatus,gender,income,...,number of open complaints,number of policies,policy type,policy,renew offer type,sales channel,total claim amount,vehicle class,vehicle size,month
0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,48029,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,2
1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,0,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,1
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,2
3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,1
4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,...,0.0,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,1


In [33]:
df4.to_csv("Week2_2023.06.06_Lab1_marketing_customer_analysis_clean.csv")

## Exercise 2: Structuring Data

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total_claim_amount to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

3. You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table. 

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [13]:
print(df4["sales channel"].unique())

['Agent' 'Call Center' 'Branch' 'Web']


In [14]:
# Your code goes here
pivotRevenueBySalesChannel = pd.pivot_table(df4, index="sales channel") #, columns=['Agent', 'Call Center', 'Branch', 'Web'])
pivotRevenueBySalesChannel1 = pd.pivot_table(df4, index="sales channel")["monthly premium auto"]
pivotRevenueBySalesChannel2 = pd.pivot_table(df4, index="sales channel")["total claim amount"]

In [15]:
pivotRevenueBySalesChannel1 = pivotRevenueBySalesChannel1.round(2)
pivotRevenueBySalesChannel2 = pivotRevenueBySalesChannel2.round(2)

print(pivotRevenueBySalesChannel1)
print(pivotRevenueBySalesChannel2)

# agents sell the most, branch or call center is least effective

sales channel
Agent          93.75
Branch         92.97
Call Center    92.47
Web            93.18
Name: monthly premium auto, dtype: float64
sales channel
Agent          439.27
Branch         430.58
Call Center    432.79
Web            434.56
Name: total claim amount, dtype: float64


In [303]:
# which months had the highest number of complaints by policy type category

In [304]:
pivotMonthOfMaxNumberOfOpenComplaints1 = pd.pivot_table(df4, index="month")["number of open complaints"]

In [305]:
pivotMonthOfMaxNumberOfOpenComplaints2 = pd.pivot_table(df4, values="number of open complaints", columns="policy type", index="month")

In [306]:
print(pivotMonthOfMaxNumberOfOpenComplaints1)
print(pivotMonthOfMaxNumberOfOpenComplaints2)

# only January and February? effective to date correct?

month
1    0.388353
2    0.379548
Name: number of open complaints, dtype: float64
policy type  Corporate Auto  Personal Auto  Special Auto
month                                                   
1                  0.352292       0.399951      0.366812
2                  0.351852       0.382543      0.471503


In [ ]:
# pd.wide_to_long(df4, stubnames, i, j, sep='', suffix='\\d+')